In [1]:
# Dependencies
import json
import os
import pandas as pd

In [2]:
#open json
object_path = 'renamed_all_data.json'
with open(object_path) as jsonfile:
       data_json = json.load(jsonfile)
df_from_json = pd.DataFrame(data_json)

In [3]:
df_from_json

,NCTId,ArmGroupInterventionName,ArmGroupType,BaselineMeasureDispersionType,BaselineMeasureParamType,CollaboratorClass,ConditionBrowseBranchAbbrev,ConditionBrowseLeafId,ConditionMeshId,DesignAllocation,...,IsFDARegulatedDevice,IsFDARegulatedDrug,LeadSponsorClass,OverallStatus,PatientRegistry,Phase,StdAge,StudyType,ConditionAncestorId,SamplingMethod
0,NCT04371159,Device: Velieve U.S.,Experimental,0,0,0,"[BC01, All, BXS]","[M8866, M4951, M15885, M15902]",D000014552,N/A,...,Yes,No,INDUSTRY,Completed,0,Not Applicable,"[Adult, Older Adult]",Interventional,"[D000007239, D000014570]",0
1,NCT04474496,Other: Assessing impact of COVID19,0,0,0,OTHER,"[BC18, BC19, All]","[M5698, M5702]",0,0,...,No,No,OTHER,Enrolling by invitation,No,0,"[Adult, Older Adult]",Observational,0,Non-Probability Sample
2,NCT02801266,Behavioral: Evidence informed birth control co...,"[Experimental, No Intervention]",0,0,OTHER,0,0,0,Randomized,...,0,0,OTHER,Completed,0,Not Applicable,Adult,Interventional,0,0
3,NCT01599390,Other: Data collection,0,0,0,INDUSTRY,"[BC02, BC08, All, BC01]","[M8878, M8866, M4951, M11485, M16105, M13732, ...",D000007251,0,...,0,0,INDUSTRY,Completed,0,0,"[Child, Adult, Older Adult]",Observational,"[D000009976, D000012327, D000014777, D00001214...",Non-Probability Sample
4,NCT00312702,[Biological: Falciparum Malaria Protein 11 wit...,"[Experimental, Experimental]",Standard Deviation,"[Mean, Count of Participants, Count of Partici...","[INDUSTRY, OTHER, FED]","[BC03, All, BC01, Rare]","[M17716, M9863, M8866, M4951, M12971, M11768, ...","[D000008288, D000016778]",Randomized,...,0,0,FED,Completed,0,"[Phase 1, Phase 2]",Adult,Interventional,"[D000011528, D000010272]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138972,NCT00002678,"[Drug: melphalan, Drug: prednisone, Drug: dexa...","[Active Comparator, Active Comparator]",0,0,0,"[BC04, BC14, BC15, BC20, All, Rare]","[M10641, M26175, M12427, M10898, M20561, M3641...","[D000009101, D000054219, D000010954]",Randomized,...,No,Yes,NETWORK,Completed,0,Phase 3,"[Adult, Older Adult]",Interventional,"[D000009370, D000009369, D000020141, D00001465...",0
138973,NCT04183179,"Behavioral: Happy Family, Healthy Kids Program",Experimental,0,0,0,"[BC18, BC23, All, BXM]","[M11284, M14688, M28742, M23894, M11267, M2477...",D000063766,N/A,...,No,No,OTHER,Not yet recruiting,0,"[Phase 1, Phase 2]",Child,Interventional,"[D000009765, D000044343, D000009748, D00005017...",0
138974,NCT03278743,[Other: Cognitive performance and cognitive de...,0,0,0,0,0,0,0,0,...,No,No,OTHER,Completed,No,0,Older Adult,Observational,0,Non-Probability Sample
138975,NCT02740686,"[Other: Sample collection, Other: Sample colle...",0,0,0,OTHER_GOV,"[BC23, All, BC08]","[M8876, M9751, M9753, M22033, M13560]","[D000008173, D000029424]",0,...,0,0,OTHER,Completed,No,0,"[Adult, Older Adult]",Observational,"[D000008171, D000012140]",Probability Sample


In [4]:
# explodeTable function - to open a column from the list and create a new table from it
def explodeTable(df_name, column_name):
    tb = df_name[['NCTId', column_name]].copy()
    explode_tb = tb.explode(column_name)
    explode_tb = explode_tb.replace(0, "-")
    explode_tb = explode_tb.sort_values(by=[column_name], ascending=False)
    fr = len(explode_tb) - explode_tb[column_name].str.contains('-', na=False).sum()
    print('feature_row', fr)
    feature_tb = explode_tb.head(fr)
    #.rename(columns={column_name:'features'})

    return feature_tb

In [5]:
# extract "OverallStatus" value 
status_tb = df_from_json[['NCTId', 'OverallStatus']].copy()
status_tb = status_tb.explode('OverallStatus')
status_tb.sort_values('NCTId', inplace=True)
status_tb.reset_index(drop=True, inplace=True)
status_tb

,NCTId,OverallStatus
0,NCT00000102,Completed
1,NCT00000104,Completed
2,NCT00000105,Terminated
3,NCT00000106,Unknown status
4,NCT00000107,Completed
...,...,...
138972,NCT04609995,Not yet recruiting
138973,NCT04610164,Enrolling by invitation
138974,NCT04610515,Not yet recruiting
138975,NCT04610658,Recruiting


In [6]:
# extract "Phase" value
phase_tb = df_from_json[['NCTId', 'Phase']].copy()
phase_tb = phase_tb.explode('Phase')
phase_tb = phase_tb.replace({'Not Applicable':'Phase_n/a',0:'Phase_n/a','Early Phase 1': 'Phase_1', 'Phase 1': 'Phase_1', 'Phase 2': 'Phase_2', 'Phase 3': 'Phase_3', 'Phase 4': 'Phase_4'})
phase_tb

,NCTId,Phase
0,NCT04371159,Phase_n/a
1,NCT04474496,Phase_n/a
2,NCT02801266,Phase_n/a
3,NCT01599390,Phase_n/a
4,NCT00312702,Phase_1
...,...,...
138973,NCT04183179,Phase_1
138973,NCT04183179,Phase_2
138974,NCT03278743,Phase_n/a
138975,NCT02740686,Phase_n/a


In [7]:
# create table to split the Phases
phase_crosstb = pd.crosstab(index=phase_tb['NCTId'], columns = phase_tb['Phase']).reset_index()
phase_crosstb.sort_values('NCTId', inplace=True)
phase_crosstb = phase_crosstb.rename(columns={'NCTId':'Id'})
phase_crosstb

Phase,Id,Phase_1,Phase_2,Phase_3,Phase_4,Phase_n/a
0,NCT00000102,1,1,0,0,0
1,NCT00000104,0,0,0,0,1
2,NCT00000105,0,0,0,0,1
3,NCT00000106,0,0,0,0,1
4,NCT00000107,0,0,0,0,1
...,...,...,...,...,...,...
138972,NCT04609995,0,0,0,0,1
138973,NCT04610164,0,0,1,0,0
138974,NCT04610515,0,0,0,0,1
138975,NCT04610658,1,1,0,0,0


# create a new table with "Status"
df = df_from_json.copy()
df= df.drop(columns=['OverallStatus', 'Phase'])
df.insert(1,'Status',status_tb['OverallStatus'])
df.sort_values('NCTId', inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [8]:
# attach the Phases to the new table
tb00 = pd.concat([status_tb,phase_crosstb], axis=1, sort=False)
tb00.drop('Id', axis=1, inplace=True)
tb00

,NCTId,OverallStatus,Phase_1,Phase_2,Phase_3,Phase_4,Phase_n/a
0,NCT00000102,Completed,1,1,0,0,0
1,NCT00000104,Completed,0,0,0,0,1
2,NCT00000105,Terminated,0,0,0,0,1
3,NCT00000106,Unknown status,0,0,0,0,1
4,NCT00000107,Completed,0,0,0,0,1
...,...,...,...,...,...,...,...
138972,NCT04609995,Not yet recruiting,0,0,0,0,1
138973,NCT04610164,Enrolling by invitation,0,0,1,0,0
138974,NCT04610515,Not yet recruiting,0,0,0,0,1
138975,NCT04610658,Recruiting,1,1,0,0,0


In [10]:
tb00.to_csv('CSVTables/Cross_Tb31.csv')